In [1]:
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install eli5==0.13.0
import matplotlib.pyplot as plt
# 한글 폰트 사용을 위한 라이브러리입니다.
plt.rcParams['font.family'] = 'AppleGothic'

In [3]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = 'train.csv'
test_path  = 'test.csv'
bus_path  = 'bus_feature.csv'
subway_path  = 'subway_feature.csv'

dt_train = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_bus=pd.read_csv(bus_path)
dt_subway=pd.read_csv(subway_path)

In [5]:
print('Train data shape : ', dt_train.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [6]:
print('bus data shape : ', dt_bus.shape, 'subway data shape : ', dt_subway.shape)

bus data shape :  (12584, 6) subway data shape :  (768, 5)


In [7]:
dt_bus=dt_bus.rename(columns={'X좌표':'경도','Y좌표':'위도'})
dt_bus=dt_bus[['노드 ID','정류소번호','정류소명','위도','경도','정류소 타입']]
dt_bus=dt_bus.rename(columns={'정류소 타입':'정류소타입'})

In [8]:
dt_test['계약년월'].value_counts()

202308    3864
202307    3695
202309    1713
Name: 계약년월, dtype: int64

In [9]:
dt_test.head()

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,직영,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,위탁,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0


In [10]:
dt_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역사명     768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [11]:
dt_subway.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [12]:
dt_subway.value_counts()

역사_ID  역사명   호선   위도         경도        
150    서울역   1호선  37.556228  126.972135    1
151    시청    1호선  37.565715  126.977088    1
2732   강남구청  7호선  37.517179  127.041255    1
2733   학동    7호선  37.514229  127.031656    1
2734   논현    7호선  37.511093  127.021415    1
                                          ..
1708   금정    경부선  37.372221  126.943429    1
1709   군포    경부선  37.353560  126.948462    1
1710   의왕    경부선  37.320852  126.948217    1
1711   성균관대  경부선  37.300349  126.970750    1
9996   미사    5호선  37.560927  127.193877    1
Length: 768, dtype: int64

In [13]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt_train['is_test'] = 0
dt_test['is_test'] = 1
dt_all = pd.concat([dt_train, dt_test])     # 하나의 데이터로 만들어줍니다.

In [14]:
dt_all['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

0    1118822
1       9272
Name: is_test, dtype: int64

In [15]:
dt_all.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test'],
      dtype='object')

In [16]:
selected_columns=['시군구','아파트명','전용면적(㎡)','계약년월','건축년도','도로명','k-세대타입(분양형태)','k-건설사(시공사)',
                  '좌표X','좌표Y','target', 'is_test']
dt_all=dt_all[selected_columns]

In [17]:
dt_all.columns = dt_all.columns.str.replace('k-', '', regex=False)
dt_all=dt_all.rename(columns={'좌표X':'경도','좌표Y':'위도'})

In [18]:
dt_all.columns

Index(['시군구', '아파트명', '전용면적(㎡)', '계약년월', '건축년도', '도로명', '세대타입(분양형태)',
       '건설사(시공사)', '경도', '위도', 'target', 'is_test'],
      dtype='object')

In [19]:
import re

# '구' 추출 (구로 끝나는 단위)
dt_all['구'] = dt_all['시군구'].str.extract(r'([가-힣]+구)')

# '동' 추출 (동으로 끝나는 단위)  
dt_all['동'] = dt_all['시군구'].str.extract(r'([가-힣]+동)')

# 원본 '시군구' 컬럼 삭제 (필요시)
dt_all = dt_all.drop('시군구', axis=1)

In [20]:
!pip install geopy

In [21]:
!pip install scipy

In [22]:
import numpy as np

# 1번: 년도와 월 분리
dt_all['계약년도'] = dt_all['계약년월'] // 100
dt_all['계약월'] = dt_all['계약년월'] % 100

# 3번: 상대적 시간 (경과개월수) 
base_date = 200701  # 2007년 1월 기준 (데이터 시작점)
dt_all['경과개월수'] = (dt_all['계약년도'] - 2007) * 12 + (dt_all['계약월'] - 1)

# 4번: 계절/분기 정보
# 분기 정보
dt_all['계약분기'] = ((dt_all['계약월'] - 1) // 3) + 1

# 계절 정보 (부동산 시장 관점)
season_mapping = {
    12: '겨울', 1: '겨울', 2: '겨울',      # 비수기
    3: '봄', 4: '봄', 5: '봄',             # 성수기 (이사철)
    6: '여름', 7: '여름', 8: '여름',        # 비수기
    9: '가을', 10: '가을', 11: '가을'        # 성수기 (가을 이사철)
}
dt_all['계약계절'] = dt_all['계약월'].map(season_mapping)

# 부동산 성수기/비수기 구분
peak_months = [3, 4, 5, 9, 10, 11]  # 봄, 가을 이사철
dt_all['성수기여부'] = dt_all['계약월'].isin(peak_months).astype(int)

# 결과 확인
print("=== 계약년월 피쳐 엔지니어링 결과 ===")
print(f"원본 계약년월 예시: {dt_all['계약년월'].head().tolist()}")
print(f"계약년도 범위: {dt_all['계약년도'].min()} ~ {dt_all['계약년도'].max()}")
print(f"계약월 범위: {dt_all['계약월'].min()} ~ {dt_all['계약월'].max()}")

print("\n=== 월별 계약 분포 ===")
monthly_dist = dt_all['계약월'].value_counts().sort_index()
for month, count in monthly_dist.items():
    season = season_mapping[month]
    peak = "성수기" if month in peak_months else "비수기"
    print(f"{month}월: {count:,}개 ({season}, {peak})")

print("\n=== 계절별 계약 분포 ===")
seasonal_dist = dt_all['계약계절'].value_counts()
for season, count in seasonal_dist.items():
    percentage = count / len(dt_all) * 100
    print(f"{season}: {count:,}개 ({percentage:.1f}%)")

print("\n=== 성수기/비수기 분포 ===")
peak_dist = dt_all['성수기여부'].value_counts()
for is_peak, count in peak_dist.items():
    label = "성수기" if is_peak == 1 else "비수기"
    percentage = count / len(dt_all) * 100
    print(f"{label}: {count:,}개 ({percentage:.1f}%)")

print("\n=== 추가된 새로운 칼럼들 ===")
new_columns = ['계약년도', '계약월', '경과개월수', '계약분기', '계약계절', '성수기여부']
print("새로 생성된 칼럼:", new_columns)
print("총 추가 칼럼 수:", len(new_columns))

# 원본 계약년월 칼럼 제거 (이제 불필요)
print("\n원본 '계약년월' 칼럼 제거...")
dt_all = dt_all.drop(['계약년월'], axis=1)
print("완료!")

=== 계약년월 피쳐 엔지니어링 결과 ===
원본 계약년월 예시: [201712, 201712, 201712, 201801, 201801]
계약년도 범위: 2007 ~ 2023
계약월 범위: 1 ~ 12

=== 월별 계약 분포 ===
1월: 87,017개 (겨울, 비수기)
2월: 89,646개 (겨울, 비수기)
3월: 106,936개 (봄, 성수기)
4월: 95,160개 (봄, 성수기)
5월: 99,429개 (봄, 성수기)
6월: 112,437개 (여름, 비수기)
7월: 109,255개 (여름, 비수기)
8월: 100,735개 (여름, 비수기)
9월: 81,180개 (가을, 성수기)
10월: 92,922개 (가을, 성수기)
11월: 77,808개 (가을, 성수기)
12월: 75,569개 (겨울, 비수기)

=== 계절별 계약 분포 ===
여름: 322,427개 (28.6%)
봄: 301,525개 (26.7%)
겨울: 252,232개 (22.4%)
가을: 251,910개 (22.3%)

=== 성수기/비수기 분포 ===
비수기: 574,659개 (50.9%)
성수기: 553,435개 (49.1%)

=== 추가된 새로운 칼럼들 ===
새로 생성된 칼럼: ['계약년도', '계약월', '경과개월수', '계약분기', '계약계절', '성수기여부']
총 추가 칼럼 수: 6

원본 '계약년월' 칼럼 제거...
완료!


In [23]:
dt_all = dt_all.drop(['계약계절', '성수기여부','경과개월수','계약분기','계약월'], axis=1)

In [24]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017
1,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,123500.0,0,강남구,개포동,2017
2,개포6차우성,54.98,1987,언주로 3,분양,우성건설,127.05721,37.476763,91500.0,0,강남구,개포동,2017
3,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,130000.0,0,강남구,개포동,2018
4,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,117000.0,0,강남구,개포동,2018


In [25]:
# 완전 벡터화 - 가장 빠름
def add_subway_ultra_fast(dt_all, dt_subway, threshold=0.005):
    """
    완전 벡터화된 버전 - 훨씬 빠름
    """
    print("벡터화 계산 시작...")
    
    # numpy 배열로 변환
    apt_coords = dt_all[['위도', '경도']].values
    subway_coords = dt_subway[['위도', '경도']].values
    
    # 브로드캐스팅으로 한번에 계산
    lat_diff = abs(apt_coords[:, 0:1] - subway_coords[:, 0])  # (N_apt, N_subway)
    lon_diff = abs(apt_coords[:, 1:2] - subway_coords[:, 1])  # (N_apt, N_subway)
    
    # 임계값 내 지하철역이 있는지 확인
    nearby = ((lat_diff < threshold) & (lon_diff < threshold)).any(axis=1)
    
    dt_all['지하철유무'] = nearby.astype(int)
    print("완료!")
    return dt_all

In [26]:
add_subway_ultra_fast(dt_all, dt_subway, threshold=0.005)

벡터화 계산 시작...
완료!


,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,1987,언주로 3,분양,우성건설,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,신내우디안1단지,84.65,2014,신내역로1길 85,기타,한신공영(주),127.10672,37.618870,NaN,1,중랑구,신내동,2023,0
9268,신내우디안1단지,84.62,2014,신내역로1길 85,기타,한신공영(주),127.10672,37.618870,NaN,1,중랑구,신내동,2023,0
9269,신내우디안1단지,101.65,2014,신내역로1길 85,기타,한신공영(주),127.10672,37.618870,NaN,1,중랑구,신내동,2023,0
9270,신내우디안1단지,84.94,2014,신내역로1길 85,기타,한신공영(주),127.10672,37.618870,NaN,1,중랑구,신내동,2023,0


In [27]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,1987,언주로 3,분양,우성건설,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0


In [28]:
# 데이터프레임이 손상되었는지 확인
print(dt_all.shape)

(1128094, 14)


In [29]:
dt_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 14 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1125958 non-null  object 
 1   전용면적(㎡)     1128094 non-null  float64
 2   건축년도        1128094 non-null  int64  
 3   도로명         1128094 non-null  object 
 4   세대타입(분양형태)  251969 non-null   object 
 5   건설사(시공사)    250457 non-null   object 
 6   경도          251862 non-null   float64
 7   위도          251862 non-null   float64
 8   target      1118822 non-null  float64
 9   is_test     1128094 non-null  int64  
 10  구           1128094 non-null  object 
 11  동           1123470 non-null  object 
 12  계약년도        1128094 non-null  int64  
 13  지하철유무       1128094 non-null  int64  
dtypes: float64(4), int64(4), object(6)
memory usage: 129.1+ MB


In [30]:
dt_all['지하철유무'].value_counts()

0    976645
1    151449
Name: 지하철유무, dtype: int64

In [31]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도,지하철유무
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0
1,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0
2,개포6차우성,54.98,1987,언주로 3,분양,우성건설,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0
3,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0
4,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0


In [32]:
dt_all['도로명'].nunique()

9245

In [33]:
dt_all['도로명']

0           언주로 3
1           언주로 3
2           언주로 3
3           언주로 3
4           언주로 3
          ...    
9267    신내역로1길 85
9268    신내역로1길 85
9269    신내역로1길 85
9270    신내역로1길 85
9271    신내역로1길 85
Name: 도로명, Length: 1128094, dtype: object

In [34]:
# 좋은_도로 피처만 추가하는 간단한 코드

# 1. 프리미엄 도로 리스트
premium_roads = ['강남대로', '테헤란로', '도산대로', '논현로', '압구정로', '봉은사로', 
                '언주로', '반포대로', '서초대로', '잠실로', '올림픽로', '한강대로', 
                '여의대로', '을지로', '종로', '세종대로', '퇴계로', '소공로', '명동길']

# 2. 주요 도로 리스트 (확장)
major_roads_extended = [
    # 강남권 추가
    '선릉로', '역삼로', '학동로', '신사역로', '압구정역로', '청담로', '삼성로',
    '영동대로', '선릉역로', '개포로', '일원로', '수서로',
    
    # 서초/반포 추가  
    '방배로', '사평대로', '동작대로', '현충로', '서래로',
    
    # 송파/잠실 추가
    '백제고분로', '송파대로', '양재대로', '가락로', '문정로',
    
    # 여의도/마포 추가
    '마포대로', '양화로', '서강대로', '홍익로', '합정로',
    
    # 강북 주요 도로
    '동호로', '장충단로', '충무로', '명륜길', '대학로', '성균관로',
    '창경궁로', '돈화문로', '인사동길', '삼일대로', '청계천로',
    
    # 용산/이태원
    '이태원로', '한남대로', '보광로', '소월로',
    
    # 영등포/구로
    '영등포로', '당산로', '선유로', '경인로',
    
    # 기타 주요 간선도로
    '강변북로', '내부순환로', '외곽순환고속도로', '경부고속도로'
]

# 3. 전체 좋은 도로 리스트 (프리미엄 + 주요)
all_good_roads = premium_roads + major_roads_extended

# 4. 도로명 컬럼 (실제 컬럼명으로 수정 필요)
road_column = '도로명'  # '도로명주소', '주소' 등으로 변경

# 5. 좋은_도로 피처 생성
print("좋은_도로 피처 생성 중...")
dt_all['좋은_도로'] = dt_all[road_column].astype(str).str.contains(
    '|'.join(all_good_roads), na=False
).astype(int)

# 6. 결과 확인
total_count = len(dt_all)
good_road_count = dt_all['좋은_도로'].sum()
good_road_ratio = good_road_count / total_count

print(f"\n=== 좋은_도로 피처 생성 완료 ===")
print(f"전체 데이터: {total_count:,}개")
print(f"좋은_도로 = 1: {good_road_count:,}개 ({good_road_ratio:.1%})")
print(f"좋은_도로 = 0: {total_count - good_road_count:,}개 ({1-good_road_ratio:.1%})")

# 7. 데이터 타입 확인
print(f"\n피처 정보:")
print(f"- 컬럼명: 좋은_도로")
print(f"- 데이터 타입: {dt_all['좋은_도로'].dtype}")
print(f"- 값 분포: {dict(dt_all['좋은_도로'].value_counts().sort_index())}")

print("\n✅ '좋은_도로' 피처가 성공적으로 추가되었습니다!")
print("   이제 모델 학습에 사용할 수 있습니다.")

좋은_도로 피처 생성 중...

=== 좋은_도로 피처 생성 완료 ===
전체 데이터: 1,128,094개
좋은_도로 = 1: 168,063개 (14.9%)
좋은_도로 = 0: 960,031개 (85.1%)

피처 정보:
- 컬럼명: 좋은_도로
- 데이터 타입: int64
- 값 분포: {0: 960031, 1: 168063}

✅ '좋은_도로' 피처가 성공적으로 추가되었습니다!
   이제 모델 학습에 사용할 수 있습니다.


In [35]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1
1,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1
2,개포6차우성,54.98,1987,언주로 3,분양,우성건설,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1
3,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1
4,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1


In [36]:
dt_all['구'].nunique()

25

In [37]:
dt_all['동'].nunique()

250

In [38]:
dt_all['아파트명'].nunique()

6549

In [39]:
dt_all['도로명'].nunique()

9245

In [40]:
import numpy as np

# 주요 지하철역 좌표 (위도, 경도 순서)
major_stations = {
    # 강남권 (4개)
    '강남역': (37.4979, 127.0276),
    '역삼역': (37.5000, 127.0364), 
    '선릉역': (37.5172, 127.0286),
    '압구정역': (37.5270, 127.0276),
    
    # 서초/반포권 (3개)
    '교대역': (37.4932, 127.0145),
    '사당역': (37.4766, 126.9816),
    '서초역': (37.4837, 127.0103),
    
    # 잠실/송파권 (3개)  
    '잠실역': (37.5133, 127.1028),
    '종합운동장역': (37.5116, 127.0736),
    '자양역': (37.5349, 127.0689),
    
    # 여의도/마포권 (3개)
    '여의도역': (37.5219, 126.9245),
    '홍대입구역': (37.5571, 126.9234),
    '신촌역': (37.5548, 126.9366),
    
    # 중심가 (2개)
    '종로3가역': (37.5735, 126.9788),
    '명동역': (37.5665, 126.9780),
    
    # 북부권 (3개)
    '노원역': (37.6542, 127.0615),
    '수유역': (37.6374, 127.0256),
    '미아사거리역': (37.6134, 127.0302),
    
    # 동부/서부 (2개)
    '건대입구역': (37.5405, 127.0704),
    '목동역': (37.5267, 126.8753)
}

print("주요 지하철역 개수:", len(major_stations))

# 역 좌표들을 numpy 배열로 변환
stations_coords = np.array(list(major_stations.values()))
print("역 좌표 배열 크기:", stations_coords.shape)

# 모든 아파트의 위도, 경도 좌표
apt_coords = dt_all[['위도', '경도']].values
print("아파트 좌표 배열 크기:", apt_coords.shape)

print("\n거리 계산 시작...")

# 브로드캐스팅으로 모든 아파트-역 간 거리 계산 (벡터화)
# apt_coords: (1000000, 2), stations_coords: (20, 2)
# 결과: (1000000, 20) 거리 행렬
distances = np.sqrt(((apt_coords[:, None, :] - stations_coords[None, :, :]) ** 2).sum(axis=2))

print("거리 계산 완료!")

# 각 아파트에서 가장 가까운 역까지의 거리
min_distances = distances.min(axis=1)

print("최단 거리 계산 완료!")

# 거리 기반 5단계 점수 부여
dt_all['지하철접근성_5단계'] = np.select([
    min_distances <= 0.005,  # 5점: 매우 가까움 (약 500m 이내)
    min_distances <= 0.010,  # 4점: 가까움 (약 1km 이내) 
    min_distances <= 0.020,  # 3점: 보통 (약 2km 이내)
    min_distances <= 0.030   # 2점: 멀음 (약 3km 이내)
], [5, 4, 3, 2], default=1)  # 1점: 매우 멀음 (3km 초과)

print("새로운 칼럼 생성 완료!")

# 결과 확인
print("\n=== 지하철접근성 5단계 분포 ===")
score_distribution = dt_all['지하철접근성_5단계'].value_counts().sort_index()
for score, count in score_distribution.items():
    percentage = count / len(dt_all) * 100
    print(f"점수 {score}: {count:,}개 ({percentage:.1f}%)")

print(f"\n총 데이터: {len(dt_all):,}개")

# 기존 지하철근접성과 비교
print("\n=== 기존 vs 새로운 접근성 비교 ===")
comparison = dt_all.groupby(['지하철유무', '지하철접근성_5단계']).size().unstack(fill_value=0)
print(comparison)

주요 지하철역 개수: 20
역 좌표 배열 크기: (20, 2)
아파트 좌표 배열 크기: (1128094, 2)

거리 계산 시작...
거리 계산 완료!
최단 거리 계산 완료!
새로운 칼럼 생성 완료!

=== 지하철접근성 5단계 분포 ===
점수 1: 981,731개 (87.0%)
점수 2: 68,986개 (6.1%)
점수 3: 57,400개 (5.1%)
점수 4: 14,593개 (1.3%)
점수 5: 5,384개 (0.5%)

총 데이터: 1,128,094개

=== 기존 vs 새로운 접근성 비교 ===
지하철접근성_5단계       1      2      3     4     5
지하철유무                                       
0           929658  26501  15042  5444     0
1            52073  42485  42358  9149  5384


In [41]:
dt_all.head(1)

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),건설사(시공사),경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계
0,개포6차우성,79.97,1987,언주로 3,분양,우성건설,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1


In [42]:
dt_all['건설사(시공사)'].value_counts()

삼성물산         14259
대우건설         11549
현대건설         10575
대한주택공사        7665
GS건설          7015
             ...  
두산               5
신영건설             3
자이에스앤디(주)        2
혜림건설             1
삼익건설(주)          1
Name: 건설사(시공사), Length: 344, dtype: int64

In [43]:
# 상위 5개만 대형으로 분류
top_5_builders = dt_all['건설사(시공사)'].value_counts().head(5).index
dt_all['대형건설사여부'] = dt_all['건설사(시공사)'].isin(top_5_builders).astype(int)
dt_all = dt_all.drop(['건설사(시공사)'], axis=1)

In [44]:
dt_all['대형건설사여부'].value_counts()

0    1077031
1      51063
Name: 대형건설사여부, dtype: int64

In [45]:
print(dt_all['동'].value_counts())

강동     62425
상계동    45232
성동     39831
중계동    26524
신정동    24240
       ...  
누상동        8
주성동        5
연지동        4
입정동        3
구수동        3
Name: 동, Length: 250, dtype: int64


In [46]:

dong_counts = dt_all['동'].value_counts()
print("\n=== 프리미엄 동 후보들 확인 ===")
premium_candidates = ['대치동', '도곡동', '압구정동', '청담동', '반포동', 
                     '역삼동', '논현동', '서초동', '잠원동', '목동', 
                     '중계동', '상계동', '잠실동', '신천동', 
                     '여의도동', '한남동', '이태원동', '개포동']

for dong in premium_candidates:
    count = dong_counts.get(dong, 0)
    print(f"{dong}: {count:,}개")

# 방법 4: 데이터가 많은 상위 50개 동 확인
print("\n=== 데이터 상위 50개 동 ===")
print(dong_counts.head(50).to_string())


=== 프리미엄 동 후보들 확인 ===
대치동: 10,342개
도곡동: 9,610개
압구정동: 4,795개
청담동: 4,287개
반포동: 11,418개
역삼동: 8,246개
논현동: 3,310개
서초동: 16,035개
잠원동: 10,648개
목동: 17,393개
중계동: 26,524개
상계동: 45,232개
잠실동: 14,686개
신천동: 8,703개
여의도동: 5,249개
한남동: 2,704개
이태원동: 826개
개포동: 12,309개

=== 데이터 상위 50개 동 ===
강동      62425
상계동     45232
성동      39831
중계동     26524
신정동     24240
창동      23222
구로동     22941
봉천동     20708
목동      17393
월계동     17176
서초동     16035
공릉동     15863
잠실동     14686
신림동     14179
화곡동     14084
신월동     13280
미아동     13079
가락동     13065
신내동     12746
방화동     12497
개포동     12309
정릉동     12219
방학동     11449
반포동     11418
시흥동     11372
개봉동     11201
쌍문동     11174
하계동     11042
길음동     10992
상도동     10967
잠원동     10648
당산동     10576
등촌동     10535
대치동     10342
장안동     10203
사당동     10136
방배동      9998
염창동      9855
가양동      9651
돈암동      9627
도곡동      9610
신천동      8703
자양동      8590
진관동      8580
면목동      8460
홍제동      8337
답십리동     8271
역삼동      8246
신당동      8231
신길동      7993


In [47]:
dong_premium_dict = {
    # 최고 프리미엄 (5점) - 교육/부촌 1등급
    '대치동': 5, '도곡동': 5, '압구정동': 5, '청담동': 5, '반포동': 5,'서초동':5, 
    '방배동':5,'역삼동': 5, '논현동': 5, '개포동': 5,'한남동': 5,'성수동':5,
    
    # 중간 프리미엄 (4점) - 교육/비즈니스/신도시 핵심
    '잠원동': 4, '목동': 4, '중계동': 4, '상계동': 4, '잠실동': 4, '신천동': 4,'여의도동': 4,'강동':4,
    # 프리미엄 (3점)
    
    '중계동':3, '상계동':3, '관악구':3, '신림동':3, '봉천동':3, '사당동':3,'신촌동':3,'화양동':3, '자양동':3, '구의동':3
}

dt_all['동프리미엄점수'] = dt_all['동'].map(dong_premium_dict).fillna(1)

In [48]:
dt_all['동프리미엄점수'] = dt_all['동프리미엄점수'].astype(int)

In [49]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5
1,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,5
2,개포6차우성,54.98,1987,언주로 3,분양,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,5
3,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,5
4,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,5


In [50]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,세대타입(분양형태),경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5
1,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,5
2,개포6차우성,54.98,1987,언주로 3,분양,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,5
3,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,5
4,개포6차우성,79.97,1987,언주로 3,분양,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,5


In [51]:
dt_all = dt_all.drop('세대타입(분양형태)', axis=1)

In [52]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,5
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,5
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,5
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,5


In [53]:
dt_all.isnull().sum()

아파트명            2136
전용면적(㎡)            0
건축년도               0
도로명                0
경도            876232
위도            876232
target          9272
is_test            0
구                  0
동               4624
계약년도               0
지하철유무              0
좋은_도로              0
지하철접근성_5단계         0
대형건설사여부            0
동프리미엄점수            0
dtype: int64

In [54]:
dt_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 16 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1125958 non-null  object 
 1   전용면적(㎡)     1128094 non-null  float64
 2   건축년도        1128094 non-null  int64  
 3   도로명         1128094 non-null  object 
 4   경도          251862 non-null   float64
 5   위도          251862 non-null   float64
 6   target      1118822 non-null  float64
 7   is_test     1128094 non-null  int64  
 8   구           1128094 non-null  object 
 9   동           1123470 non-null  object 
 10  계약년도        1128094 non-null  int64  
 11  지하철유무       1128094 non-null  int64  
 12  좋은_도로       1128094 non-null  int64  
 13  지하철접근성_5단계  1128094 non-null  int64  
 14  대형건설사여부     1128094 non-null  int64  
 15  동프리미엄점수     1128094 non-null  int64  
dtypes: float64(4), int64(8), object(4)
memory usage: 146.3+ MB


In [55]:
dt_all.head()

,아파트명,전용면적(㎡),건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,5
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,5
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,5
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,5


In [56]:
# 1. 컬럼 타입 재분류
categorical_columns = dt_all.select_dtypes(include=['object']).columns.tolist()
numeric_columns = dt_all.select_dtypes(include=['float64', 'int64']).columns.tolist()
print("숫자형 컬럼들:", numeric_columns)
print("범주형 컬럼들:",categorical_columns)

숫자형 컬럼들: ['전용면적(㎡)', '건축년도', '경도', '위도', 'target', 'is_test', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수']
범주형 컬럼들: ['아파트명', '도로명', '구', '동']


In [57]:
dt_all[categorical_columns] = dt_all[categorical_columns].fillna('NULL')

In [58]:
dt_all[numeric_columns] = dt_all[numeric_columns].fillna(0)

In [59]:
dt_all.isnull().sum()

아파트명          0
전용면적(㎡)       0
건축년도          0
도로명           0
경도            0
위도            0
target        0
is_test       0
구             0
동             0
계약년도          0
지하철유무         0
좋은_도로         0
지하철접근성_5단계    0
대형건설사여부       0
동프리미엄점수       0
dtype: int64

In [60]:
print(dt_all.shape)

(1128094, 16)


In [61]:
dt_all = dt_all.rename(columns={"전용면적(㎡)": "전용면적"})

In [62]:
# 구별 프리미엄 점수 (5, 4, 3, 1)
district_premium = {
    5: ['강남구', '서초구', '송파구','강동구'],  # 최고급 (강남 3구)
    4: ['마포구', '용산구', '성동구', '광진구', '강동구', '영등포구','노원구'],  # 고급
    3: ['구로구', '양천구', '강서구', '종로구', '중구', '동작구', '관악구', '서대문구', '은평구', '성북구'],  # 중급
    1: ['중랑구', '동대문구','도봉구', '강북구', '금천구']  # 일반
}

def get_district_score(district_name):
    if pd.isna(district_name):
        return 1
    
    district_str = str(district_name)
    for score, districts in district_premium.items():
        for district in districts:
            if district in district_str:
                return score
    return 1

# 구 프리미엄 점수 적용
dt_all['구_프리미엄점수'] = dt_all['구'].apply(get_district_score)

# 확인
print("구 프리미엄 점수 분포:")
print(dt_all['구_프리미엄점수'].value_counts().sort_index())

구 프리미엄 점수 분포:
1    172118
3    408389
4    287509
5    260078
Name: 구_프리미엄점수, dtype: int64


In [63]:
dt_all.columns

Index(['아파트명', '전용면적', '건축년도', '도로명', '경도', '위도', 'target', 'is_test', '구',
       '동', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수',
       '구_프리미엄점수'],
      dtype='object')

In [64]:
len(dt_all.columns)

17

In [65]:
dt_all.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,구_프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5,5


In [66]:
dt_all.shape

(1128094, 17)

In [67]:
dt_all.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,구_프리미엄점수
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,2017,0,1,1,0,5,5
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,2017,0,1,1,0,5,5
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,2017,0,1,1,0,5,5
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,2018,0,1,1,0,5,5
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,2018,0,1,1,0,5,5


In [68]:
# 계약년도 피처 엔지니어링
dt_all['계약경과년수'] = dt_all['계약년도'] - 2007

In [69]:
dt_all['계약경과년수_로그'] = np.log(dt_all['계약경과년수'] + 1)
dt_all['계약경과년수_제곱'] = dt_all['계약경과년수'] ** 2

In [70]:
dt_all['매매당시_건물연령']=dt_all['계약년도']-dt_all['건축년도']

In [71]:
dt_all.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,...,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,구_프리미엄점수,계약경과년수,계약경과년수_로그,계약경과년수_제곱,매매당시_건물연령
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,...,0,1,1,0,5,5,10,2.397895,100,30
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,...,0,1,1,0,5,5,10,2.397895,100,30
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,...,0,1,1,0,5,5,10,2.397895,100,30
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,...,0,1,1,0,5,5,11,2.484907,121,31
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,...,0,1,1,0,5,5,11,2.484907,121,31


In [72]:
dt_all['매매당시_신축여부'] = (dt_all['매매당시_건물연령']<= 3).astype(int)
dt_all['매매당시_재건축대상'] = (dt_all['매매당시_건물연령'] >= 30).astype(int)

In [73]:
dt_all.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,...,지하철접근성_5단계,대형건설사여부,동프리미엄점수,구_프리미엄점수,계약경과년수,계약경과년수_로그,계약경과년수_제곱,매매당시_건물연령,매매당시_신축여부,매매당시_재건축대상
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,...,1,0,5,5,10,2.397895,100,30,0,1


In [74]:
dt_all.columns

Index(['아파트명', '전용면적', '건축년도', '도로명', '경도', '위도', 'target', 'is_test', '구',
       '동', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수',
       '구_프리미엄점수', '계약경과년수', '계약경과년수_로그', '계약경과년수_제곱', '매매당시_건물연령',
       '매매당시_신축여부', '매매당시_재건축대상'],
      dtype='object')

In [75]:
import numpy as np

# 전용면적 로그 변환
dt_all['전용면적_로그'] = np.log1p(dt_all['전용면적'])

# 전용면적 제곱 변환
dt_all['전용면적_제곱'] = dt_all['전용면적'] ** 2


In [76]:
dt_all.shape

(1128094, 25)

In [77]:
dt_all.head()

,아파트명,전용면적,건축년도,도로명,경도,위도,target,is_test,구,동,...,동프리미엄점수,구_프리미엄점수,계약경과년수,계약경과년수_로그,계약경과년수_제곱,매매당시_건물연령,매매당시_신축여부,매매당시_재건축대상,전용면적_로그,전용면적_제곱
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,124000.0,0,강남구,개포동,...,5,5,10,2.397895,100,30,0,1,4.394079,6395.2009
1,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,123500.0,0,강남구,개포동,...,5,5,10,2.397895,100,30,0,1,4.394079,6395.2009
2,개포6차우성,54.98,1987,언주로 3,127.05721,37.476763,91500.0,0,강남구,개포동,...,5,5,10,2.397895,100,30,0,1,4.024994,3022.8004
3,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,130000.0,0,강남구,개포동,...,5,5,11,2.484907,121,31,0,1,4.394079,6395.2009
4,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,117000.0,0,강남구,개포동,...,5,5,11,2.484907,121,31,0,1,4.394079,6395.2009


In [78]:
dt_all.isnull().sum()

아파트명          0
전용면적          0
건축년도          0
도로명           0
경도            0
위도            0
target        0
is_test       0
구             0
동             0
계약년도          0
지하철유무         0
좋은_도로         0
지하철접근성_5단계    0
대형건설사여부       0
동프리미엄점수       0
구_프리미엄점수      0
계약경과년수        0
계약경과년수_로그     0
계약경과년수_제곱     0
매매당시_건물연령     0
매매당시_신축여부     0
매매당시_재건축대상    0
전용면적_로그       0
전용면적_제곱       0
dtype: int64

In [79]:
# 구별 평균 좌표로 채우기
dt_all['경도'] = dt_all.groupby('구')['경도'].transform(lambda x: x.fillna(x.mean()))
dt_all['위도'] = dt_all.groupby('구')['위도'].transform(lambda x: x.fillna(x.mean()))

# 여전히 NaN이 남아있다면 동별 평균으로
dt_all['경도'] = dt_all.groupby('동')['경도'].transform(lambda x: x.fillna(x.mean()))
dt_all['위도'] = dt_all.groupby('동')['위도'].transform(lambda x: x.fillna(x.mean()))

In [80]:
dt_all.isnull().sum()

아파트명          0
전용면적          0
건축년도          0
도로명           0
경도            0
위도            0
target        0
is_test       0
구             0
동             0
계약년도          0
지하철유무         0
좋은_도로         0
지하철접근성_5단계    0
대형건설사여부       0
동프리미엄점수       0
구_프리미엄점수      0
계약경과년수        0
계약경과년수_로그     0
계약경과년수_제곱     0
매매당시_건물연령     0
매매당시_신축여부     0
매매당시_재건축대상    0
전용면적_로그       0
전용면적_제곱       0
dtype: int64

In [81]:
dt_all.columns

Index(['아파트명', '전용면적', '건축년도', '도로명', '경도', '위도', 'target', 'is_test', '구',
       '동', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수',
       '구_프리미엄점수', '계약경과년수', '계약경과년수_로그', '계약경과년수_제곱', '매매당시_건물연령',
       '매매당시_신축여부', '매매당시_재건축대상', '전용면적_로그', '전용면적_제곱'],
      dtype='object')

In [82]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = dt_all.query('is_test==0')
dt_test = dt_all.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(1118822, 24) (9272, 24)


In [83]:
dt_test.head(1)

,아파트명,전용면적,건축년도,도로명,경도,위도,target,구,동,계약년도,...,동프리미엄점수,구_프리미엄점수,계약경과년수,계약경과년수_로그,계약경과년수_제곱,매매당시_건물연령,매매당시_신축여부,매매당시_재건축대상,전용면적_로그,전용면적_제곱
0,개포6차우성,79.97,1987,언주로 3,127.05721,37.476763,0.0,강남구,개포동,2023,...,5,5,16,2.833213,256,36,0,1,4.394079,6395.2009


In [84]:
dt_test['target'] = 0

In [85]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 24 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   아파트명        1118822 non-null  object 
 1   전용면적        1118822 non-null  float64
 2   건축년도        1118822 non-null  int64  
 3   도로명         1118822 non-null  object 
 4   경도          1118822 non-null  float64
 5   위도          1118822 non-null  float64
 6   target      1118822 non-null  float64
 7   구           1118822 non-null  object 
 8   동           1118822 non-null  object 
 9   계약년도        1118822 non-null  int64  
 10  지하철유무       1118822 non-null  int64  
 11  좋은_도로       1118822 non-null  int64  
 12  지하철접근성_5단계  1118822 non-null  int64  
 13  대형건설사여부     1118822 non-null  int64  
 14  동프리미엄점수     1118822 non-null  int64  
 15  구_프리미엄점수    1118822 non-null  int64  
 16  계약경과년수      1118822 non-null  int64  
 17  계약경과년수_로그   1118822 non-null  float64
 18  계약경과년수_제곱   1118822 no

In [86]:
numeric_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        numeric_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("수치형 변수:", numeric_columns_v2)
print("범주형 변수:", categorical_columns_v2)

수치형 변수: ['전용면적', '건축년도', '경도', '위도', 'target', '계약년도', '지하철유무', '좋은_도로', '지하철접근성_5단계', '대형건설사여부', '동프리미엄점수', '구_프리미엄점수', '계약경과년수', '계약경과년수_로그', '계약경과년수_제곱', '매매당시_건물연령', '매매당시_신축여부', '매매당시_재건축대상', '전용면적_로그', '전용면적_제곱']
범주형 변수: ['아파트명', '도로명', '구', '동']


In [87]:
from sklearn.preprocessing import OneHotEncoder

# ---------------------------
# 1. 구 → 원-핫 인코딩
# ---------------------------
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
train_encoded = ohe.fit_transform(dt_train[["구"]])
test_encoded = ohe.transform(dt_test[["구"]])

new_cols = [f"구_{c}" for c in ohe.categories_[0]]
dt_train_ohe = pd.DataFrame(train_encoded, columns=new_cols, index=dt_train.index)
dt_test_ohe = pd.DataFrame(test_encoded, columns=new_cols, index=dt_test.index)

dt_train = pd.concat([dt_train.drop(columns=["구"]), dt_train_ohe], axis=1)
dt_test = pd.concat([dt_test.drop(columns=["구"]), dt_test_ohe], axis=1)

# ---------------------------
# 2. 동/아파트명/도로명 → 빈도 인코딩
# ---------------------------
for col in ["동", "아파트명", "도로명"]:
    freq_map = dt_train[col].value_counts().to_dict()
    dt_train[col + "_freq"] = dt_train[col].map(freq_map).fillna(0)
    dt_test[col + "_freq"] = dt_test[col].map(freq_map).fillna(0)
    dt_train.drop(columns=[col], inplace=True)
    dt_test.drop(columns=[col], inplace=True)

print("✅ 인코딩 완료! (구=원핫, 동/아파트명/도로명=빈도)")

✅ 인코딩 완료! (구=원핫, 동/아파트명/도로명=빈도)


In [88]:
dt_train.head()

,전용면적,건축년도,경도,위도,target,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,...,구_양천구,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,동_freq,아파트명_freq,도로명_freq
0,79.97,1987,127.05721,37.476763,124000.0,2017,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101
1,79.97,1987,127.05721,37.476763,123500.0,2017,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101
2,54.98,1987,127.05721,37.476763,91500.0,2017,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101
3,79.97,1987,127.05721,37.476763,130000.0,2018,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101
4,79.97,1987,127.05721,37.476763,117000.0,2018,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101


In [89]:
X_train=dt_train.drop(['target'],axis=1)
y_train=dt_train['target']

In [90]:
X_train.head(1)

,전용면적,건축년도,경도,위도,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,...,구_양천구,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,동_freq,아파트명_freq,도로명_freq
0,79.97,1987,127.05721,37.476763,2017,0,1,1,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101,101


In [91]:
y_train.head(1)

0    124000.0
Name: target, dtype: float64

In [92]:
X_train.shape

(1118822, 47)

In [93]:
# 계약일자(또는 계약경과년수) 기준 정렬
Xy = X_train.copy()
Xy['target'] = y_train

Xy = Xy.sort_values(by="계약경과년수", ascending=False)  # 시간 순서 반영

# 80% 학습, 20% 검증
split_idx = int(len(Xy) * 0.8)
train = Xy.iloc[:split_idx]
val   = Xy.iloc[split_idx:]

# 분리
X_tr, y_tr = train.drop(columns=['target']), train['target']
X_val, y_val = val.drop(columns=['target']), val['target']

print(X_tr.shape, X_val.shape)


(895057, 47) (223765, 47)


In [94]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 모델 정의
rf = RandomForestRegressor(
    n_estimators=400, max_depth=35, min_samples_split=5, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)

# 학습
rf.fit(X_tr, y_tr)

# 예측
pred = rf.predict(X_val)

# RMSE 계산
rmse = mean_squared_error(y_val, pred, squared=False)
print(f"랜덤포레스트 RMSE: {rmse:.2f}")


랜덤포레스트 RMSE: 7703.82


In [95]:
# 한 줄로 출력
importances = sorted(zip(X_tr.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
for feature, importance in importances[:15]:
    print(f"{feature:<20}: {importance:.4f}")

구_프리미엄점수            : 0.1238
전용면적                : 0.1226
전용면적_제곱             : 0.1195
전용면적_로그             : 0.1142
동프리미엄점수             : 0.0837
건축년도                : 0.0553
도로명_freq            : 0.0534
계약년도                : 0.0495
계약경과년수_로그           : 0.0471
계약경과년수              : 0.0446
계약경과년수_제곱           : 0.0420
동_freq              : 0.0337
위도                  : 0.0153
매매당시_건물연령           : 0.0152
아파트명_freq           : 0.0143


In [185]:
!pip install lightgbm

In [96]:
import numpy as np
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_squared_error

# 모델 정의 (기본보다 약간 튜닝)
lgbm = LGBMRegressor(
    n_estimators=5000,      # 트리 개수
    learning_rate=0.05,     # 학습률
    num_leaves=127,         # 트리 리프 개수
    min_data_in_leaf=100,   # 최소 샘플 수
    feature_fraction=0.8,   # 특성 샘플링 비율
    bagging_fraction=0.8,   # 데이터 샘플링 비율
    bagging_freq=1,
    random_state=42,
    n_jobs=-1
)

# 학습 (조기종료 포함)
lgbm.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks=[early_stopping(200)]
)

# 예측
pred = lgbm.predict(X_val, num_iteration=lgbm.best_iteration_)

# RMSE 계산
rmse = mean_squared_error(y_val, pred, squared=False)
print(f"LightGBM RMSE: {rmse:.2f}")


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Auto-choosing row-wise multi-threadi

In [98]:
import numpy as np
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_squared_error

# y 로그 변환
y_tr_log = np.log1p(y_tr)
y_val_log = np.log1p(y_val)

# 모델 정의
lgbm = LGBMRegressor(
    n_estimators=5000,
    learning_rate=0.05,
    num_leaves=127,
    min_data_in_leaf=100,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    random_state=42,
    n_jobs=-1
)

# 학습 (로그 타깃)
lgbm.fit(
    X_tr, y_tr_log,
    eval_set=[(X_val, y_val_log)],
    eval_metric="rmse",
    callbacks=[early_stopping(200)]
)

# 예측 (로그 → 원래 스케일 복원)
pred_log = lgbm.predict(X_val, num_iteration=lgbm.best_iteration_)
pred = np.expm1(pred_log)

# RMSE 계산 (원래 스케일에서)
rmse = mean_squared_error(y_val, pred, squared=False)
print(f"LightGBM RMSE (로그 변환): {rmse:.2f}")


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Auto-choosing row-wise multi-threadi

In [85]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [optuna]━━━━ 4/5 [optuna]


In [ ]:
import optuna
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 목적 함수 정의
def objective(trial):
    params = {
        "n_estimators": 5000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "random_state": 42,
        "n_jobs": -1
    }
    
    # 모델 학습 (로그 변환된 타깃 사용)
    model = LGBMRegressor(**params)
    model.fit(X_tr, np.log1p(y_tr))
    
    # 검증 RMSE (원래 스케일에서 평가)
    pred_log = model.predict(X_val)
    pred = np.expm1(pred_log)
    rmse = mean_squared_error(y_val, pred, squared=False)
    return rmse

# 탐색 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)

In [99]:
import numpy as np
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_squared_error

# Optuna 최적 파라미터
best_params = {
    'learning_rate': 0.06513375297054018,
    'num_leaves': 136,
    'min_data_in_leaf': 36,
    'feature_fraction': 0.6260664655589864,
    'bagging_fraction': 0.936495380308515,
    'bagging_freq': 3,
    'n_estimators': 5000,
    'random_state': 42,
    'n_jobs': -1
}

# 로그 변환된 타깃으로 학습
model = LGBMRegressor(**best_params)
model.fit(
    X_tr, np.log1p(y_tr),
    eval_set=[(X_val, np.log1p(y_val))],
    eval_metric="rmse",
    callbacks=[early_stopping(200)]
)

# 예측 (로그 → 원래 스케일 복원)
pred_log = model.predict(X_val, num_iteration=model.best_iteration_)
pred = np.expm1(pred_log)

# 최종 RMSE
rmse = mean_squared_error(y_val, pred, squared=False)
print(f"Final LightGBM RMSE: {rmse:.2f}")

[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6260664655589864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6260664655589864
[LightGBM] [Warning] bagging_fraction is set=0.936495380308515, subsample=1.0 will be ignored. Current value: bagging_fraction=0.936495380308515
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6260664655589864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6260664655589864
[LightGBM] [Warning] bagging_fraction is set=0.936495380308515, subsample=1.0 will be ignored. Current value: bagging_fraction=0.936495380308515
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_sampl

In [100]:
dt_test.head()

,전용면적,건축년도,경도,위도,target,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,...,구_양천구,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,동_freq,아파트명_freq,도로명_freq
0,79.9700,1987,127.057210,37.476763,0,2023,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101.0,101.0
1,108.2017,2021,127.056394,37.484892,0,2023,1,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,1.0,118.0
2,161.0000,1984,127.055990,37.483894,0,2023,1,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,159.0,159.0
3,133.4600,1984,127.055990,37.483894,0,2023,1,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,159.0,159.0
4,104.4300,1984,127.055990,37.483894,0,2023,1,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,159.0,159.0


In [102]:
dt_test=dt_test.drop(columns=["target"])

In [103]:
dt_test.head(1)

,전용면적,건축년도,경도,위도,계약년도,지하철유무,좋은_도로,지하철접근성_5단계,대형건설사여부,동프리미엄점수,...,구_양천구,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,동_freq,아파트명_freq,도로명_freq
0,79.97,1987,127.05721,37.476763,2023,0,1,1,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12218,101.0,101.0


In [104]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

# -------------------------
# 모델 정의 (Optuna 파라미터 적용)
# -------------------------
best_params = {
    'learning_rate': 0.06513375297054018,
    'num_leaves': 136,
    'min_data_in_leaf': 36,
    'feature_fraction': 0.6260664655589864,
    'bagging_fraction': 0.936495380308515,
    'bagging_freq': 3,
    'n_estimators': 5000,
    'random_state': 42,
    'n_jobs': -1,
}
model = LGBMRegressor(**best_params)

# -------------------------
# 전체 학습 데이터로 학습 (로그 변환)
# -------------------------
y_train_log = np.log1p(y_train)
model.fit(X_train, y_train_log)

# -------------------------
# Test 데이터 예측
# -------------------------
test_pred_log = model.predict(dt_test, num_iteration=model.best_iteration_)
test_pred = np.expm1(test_pred_log)
test_pred_int = np.rint(test_pred).astype(int)

# -------------------------
# 샘플 제출 파일 불러오기 + 순서 맞추기
# -------------------------
sample = pd.read_csv("sample_submission.csv")
submission = sample.copy()
submission["target"] = test_pred_int

# -------------------------
# CSV 저장
# -------------------------
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료!")

# -------------------------
# 🚨 제출 전 점검
# -------------------------
print("\n[제출 전 점검 결과]")
print("샘플 행 개수:", len(sample))
print("제출 행 개수:", len(submission))
print("샘플 컬럼:", sample.columns.tolist())
print("제출 컬럼:", submission.columns.tolist())
print("데이터 타입:\n", submission.dtypes)
print("\n제출 파일 미리보기:")
print(submission.head())


[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6260664655589864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6260664655589864
[LightGBM] [Warning] bagging_fraction is set=0.936495380308515, subsample=1.0 will be ignored. Current value: bagging_fraction=0.936495380308515
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6260664655589864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6260664655589864
[LightGBM] [Warning] bagging_fraction is set=0.936495380308515, subsample=1.0 will be ignored. Current value: bagging_fraction=0.936495380308515
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_sampl